# Práctica 3


## Importar Librerias


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Callable, List, Tuple
from tqdm import tqdm

## Dataset


### Lectura del Dataset


In [ ]:
df = pd.read_csv("diabetes_dataset.csv")
df_2019 = df[df["year"] == 2019]
df_2019.drop(columns=["year"], inplace=True)
df_2019.head()

C:\Users\jlflo\AppData\Local\Temp\ipykernel_23084\1727960749.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2019.drop(columns=["year"], inplace=True)


,gender,age,location,race:AfricanAmerican,race:Asian,race:Caucasian,race:Hispanic,race:Other,hypertension,heart_disease,smoking_history,bmi,hbA1c_level,blood_glucose_level,diabetes
73,Male,55.00,Alabama,0,0,0,1,0,0,0,No Info,27.94,6.0,158,0
335,Female,1.72,Alabama,0,0,0,1,0,0,0,No Info,16.28,6.2,100,0
337,Female,55.00,Alabama,0,1,0,0,0,0,0,never,28.30,6.0,140,0
340,Female,53.00,Alabama,0,1,0,0,0,0,0,never,35.84,6.0,155,0
341,Female,38.00,Alabama,0,1,0,0,0,0,0,never,31.91,5.8,200,0


### Discretización


In [ ]:
cols_discretizar = {
    "age": {
        "niños": [0, 9],
        "adolescentes": [10, 19],
        "jóvenes": [20, 59],
        "adulto mayor": [60],
    },
    "bmi": {
        "bajo peso": [18.49],
        "peso normal": [18.5, 24.99],
        "sobrepeso": [25, 29.99],
        "obesidad": [30],
    },
    "hbA1c_level": {"normal": [5.6], "limite": [5.7, 6.4], "sobre": [6.5]},
    "blood_glucose_level": {
        "hipoglucemia": [79],
        "normal": [80, 159],
        "elevado": [160, 199],
        "critico": [200],
    },
}


def discretizar_col(col, bins: dict):
    factor = 0
    for label, edges in bins.items():
        if len(edges) == 1:
            if factor == 0:
                if col <= edges[0]:
                    return label

            else:
                if col >= edges[0]:
                    return label
        else:
            if edges[0] <= col <= edges[1]:
                return label
        factor += 1


def discretizar_df(df: pd.DataFrame, cols_dict: dict) -> pd.DataFrame:
    df_copy = df.copy()
    for col, bins in cols_dict.items():
        df_copy[col] = df_copy[col].apply(lambda x: discretizar_col(x, bins))
    return df_copy


df_discretizado = discretizar_df(df_2019, cols_discretizar)
df_discretizado.head()

,gender,age,location,race:AfricanAmerican,race:Asian,race:Caucasian,race:Hispanic,race:Other,hypertension,heart_disease,smoking_history,bmi,hbA1c_level,blood_glucose_level,diabetes
73,Male,jóvenes,Alabama,0,0,0,1,0,0,0,No Info,sobrepeso,limite,normal,0
335,Female,niños,Alabama,0,0,0,1,0,0,0,No Info,bajo peso,limite,normal,0
337,Female,jóvenes,Alabama,0,1,0,0,0,0,0,never,sobrepeso,limite,normal,0
340,Female,jóvenes,Alabama,0,1,0,0,0,0,0,never,obesidad,limite,normal,0
341,Female,jóvenes,Alabama,0,1,0,0,0,0,0,never,obesidad,limite,critico,0


In [ ]:
vals_numericos = {
    "age": {
        "niños": 0,
        "adolescentes": 1,
        "jóvenes": 2,
        "adulto mayor": 3,
    },
    "bmi": {
        "bajo peso": 0,
        "peso normal": 1,
        "sobrepeso": 2,
        "obesidad": 3,
    },
    "hbA1c_level": {"normal": 0, "limite": 1, "sobre": 2},
    "blood_glucose_level": {
        "hipoglucemia": 0,
        "normal": 1,
        "elevado": 2,
        "critico": 3,
    },
    "gender": {"Male": 0, "Female": 1},
}


def dummy_col(col, vals: dict):
    for label, val in vals.items():
        if col == label:
            return int(val)


def dummy_df(df: pd.DataFrame, cols_discretizar: dict) -> pd.DataFrame:
    for col, bins in cols_discretizar.items():
        df[col] = df[col].apply(lambda x: dummy_col(x, bins))
    for col in df.columns:
        # Verificar si la columna necesita conversión
        necesita_conversion = False

        if df[col].dtype == "object":
            # Es texto, necesita conversión
            necesita_conversion = True
        elif df[col].dtype in ["float64", "float32", "int64", "int32"]:
            # Es numérica, verificar si tiene valores no enteros
            try:
                valores_no_nulos = df[col].dropna()
                if len(valores_no_nulos) > 0:
                    # Verificar si todos los valores son enteros
                    necesita_conversion = not all(
                        valores_no_nulos == valores_no_nulos.astype(int)
                    )
            except:
                necesita_conversion = True

        if necesita_conversion:
            # Obtener valores únicos (excluyendo NaN)
            valores_unicos = df[col].dropna().unique()

            # Crear el mapeo (ordenado alfabéticamente)
            mapeo = {
                valor: idx for idx, valor in enumerate(sorted(valores_unicos, key=str))
            }

            # Guardar en el diccionario
            cols_discretizar[col] = mapeo

            # Aplicar la conversión
            df[col] = df[col].map(mapeo)

    return df, cols_discretizar


df_dummy, vals_numericos = dummy_df(df_discretizado, vals_numericos)
df_dummy.head()

,gender,age,location,race:AfricanAmerican,race:Asian,race:Caucasian,race:Hispanic,race:Other,hypertension,heart_disease,smoking_history,bmi,hbA1c_level,blood_glucose_level,diabetes
73,0.0,2,0,0,0,0,1,0,0,0,0,2,1,1,0
335,1.0,0,0,0,0,0,1,0,0,0,0,0,1,1,0
337,1.0,2,0,0,1,0,0,0,0,0,4,2,1,1,0
340,1.0,2,0,0,1,0,0,0,0,0,4,3,1,1,0
341,1.0,2,0,0,1,0,0,0,0,0,4,3,1,3,0


## Funciones de MOEA/D


### Generar Población


In [ ]:
def generar_poblacion(df_original: pd.DataFrame, N: int = 120) -> np.ndarray:
    """
    Genera una población aleatoria a partir de una base de datos original.
    args:
        df_original (pd.DataFrame): DataFrame original del cual se generará la población.
        N (int): Tamaño de la población a generar. Default es 120.
    returns:
        np.ndarray: Arreglo con la población generada.
    """
    n_df = pd.DataFrame(columns=df_original.columns)
    for col in df_original.columns:
        n_df[col] = np.random.choice(df_original[col], size=N, replace=True)
    poblacion = np.zeros((N, 2, len(n_df.columns)))
    for i in range(len(poblacion)):
        poblacion[i][0] = np.random.choice([0, 1, 2], size=n_df.shape[1])
        poblacion[i][1] = n_df.iloc[i].values

    return poblacion

### Dan Dennis


In [ ]:
from math import comb


def calcular_H(N, M):
    """
    Calcula H dado N (puntos deseados) y M (objetivos)
    mediante búsqueda iterativa.
    """
    H = 1
    while comb(H + M - 1, M - 1) < N:
        H += 1

    puntos_generados = comb(H + M - 1, M - 1)

    if puntos_generados == N:
        return H
    else:
        print(
            f"No existe H exacto. H={H} genera {puntos_generados} puntos (cercano a {N})"
        )
        return H


def Dan_Dennis(poblacion: np.ndarray, M=4) -> np.ndarray:
    """
    Ejecuta el método de generación de población de Dan Dennis.
    args:
        poblacion (np.ndarray): Matriz original del cual se generará la población.
        M (int): Número de objetivos.
    returns:
        np.ndarray: Arreglo con los pesos.
    """
    N = poblacion.shape[0]
    H = calcular_H(N, M)
    # print(f"Usando H={H} para generar la población.")

    def generar_particiones(M, H, profundidad=0, suma_actual=0, particion_actual=[]):
        """
        Genera recursivamente todas las particiones que suman H.
        """
        if profundidad == M - 1:
            # Última posición: asignar lo que falta
            particion_actual.append(H - suma_actual)
            particiones.append(particion_actual[:])
            particion_actual.pop()
            return

        # Probar todos los valores posibles para la posición actual
        for valor in range(H - suma_actual + 1):
            particion_actual.append(valor)
            generar_particiones(
                M, H, profundidad + 1, suma_actual + valor, particion_actual
            )
            particion_actual.pop()

    particiones = []
    generar_particiones(M, H)

    # Convertir particiones a pesos
    weights = np.array(particiones, dtype=float) / H

    return weights

### Vencidario


In [ ]:
def encontrar_vecindarios(pesos: np.ndarray, T: int) -> List[np.ndarray]:
    """
    Encuentra los T vecinos más cercanos para cada vector de peso.

    args:
        pesos: Matriz de vectores de peso (n, m)
        T: Tamaño del vecindario
    returns:
        List[np.ndarray]: Lista de vecindarios (índices)
    """
    n = pesos.shape[0]
    vecindarios = []

    for i in range(n):
        distancias = np.linalg.norm(pesos - pesos[i], axis=1)
        vecinos = np.argsort(distancias)[:T]
        vecindarios.append(vecinos)

    return vecindarios

### Evaluar Individuos


In [ ]:
def comparar_individuos(
    fy: np.ndarray,
    fx_j: np.ndarray,
    lambda_j: np.ndarray,
    z: np.ndarray,
    evaluar_con_agregacion: Callable,
) -> bool:
    """
    Compara si el hijo y es mejor que x_j según la función de agregación.

    returns:
        True si y es mejor
    """

    g_y = evaluar_con_agregacion(fy, lambda_j, z)
    g_xj = evaluar_con_agregacion(fx_j, lambda_j, z)

    return g_y < g_xj

### Punto Ideal


In [ ]:
def actualizar_punto_ideal(
    z: np.ndarray,
    individuo: np.ndarray,
    objetivos: np.ndarray,
) -> np.ndarray:
    """
    Actualiza el punto ideal con los mejores valores de cada objetivo.
    args:
        z (np.ndarray): Punto ideal actual de shape (m,) donde m es número de objetivos
        individuo (np.ndarray): Individuo a evaluar
        objetivos (np.ndarray): Objetivos de un individuo
        df (pd.DataFrame): DF del dataset a evaluar
    returns:
        np.ndarray: Punto ideal actualizado
    """
    z_nuevo = z.copy()
    for i, objetivo in enumerate(objetivos):
        z_nuevo[i] = min(z_nuevo[i], objetivo)

    return z_nuevo


def inicializar_punto_ideal(
    poblacion: np.ndarray, objetivos: np.ndarray, problema: int
) -> np.ndarray:
    """
    Inicializa el punto ideal evaluando la población inicial.
    """
    if problema == 1:
        M = 3
    else:
        M = 4
    z = np.full(M, 0.0)

    for i in range(len(poblacion)):
        z = actualizar_punto_ideal(z, poblacion[i], objetivos[i])

    return z

### Funciones de Agregación


In [ ]:
def tchebycheff(f_x: np.ndarray, lambda_j: np.ndarray, z: np.ndarray) -> float:
    """
    Función de agregación de Tchebycheff.
    args:
        f_x: Vector de valores objetivos del individuo
        lambda_j: Vector de pesos
        z: Punto ideal
    returns:
        float: Valor de la función de agregación (menor es mejor)
    """
    return np.max(lambda_j * np.abs(f_x - z))

In [ ]:
def weighted_sum(f_x: np.ndarray, lambda_j: np.ndarray, z: np.ndarray) -> float:
    """
    Función de agregación de suma ponderada.

    args:
        f_x: Vector de valores objetivos del individuo
        lambda_j: Vector de pesos
        z: Punto ideal
    returns:
        float: Valor de la función de agregación
    """
    return np.sum(lambda_j * (f_x - z))

In [ ]:
def pbi(
    f_x: np.ndarray,
    lambda_j: np.ndarray,
    z: np.ndarray,
    theta: float = 5.0,
) -> float:
    """
    Función de agregación PBI (Penalty-Based Boundary Intersection).

    args:
        f_x: Vector de valores objetivos del individuo
        lambda_j: Vector de pesos
        z: Punto ideal
        theta: Parámetro de penalización
    returns:
        float: Valor de la función de agregación
    """
    diferencia = f_x - z
    norma_lambda = np.linalg.norm(lambda_j)

    if norma_lambda == 0:
        return np.inf

    d1 = np.abs(np.dot(diferencia, lambda_j)) / norma_lambda

    punto_proyeccion = z + d1 * lambda_j
    d2 = np.linalg.norm(f_x - punto_proyeccion)

    return d1 + theta * d2

## Funciones Objetivo


### Funciones principales


In [ ]:
def obtener_x_y(individuo: np.ndarray):
    """
    Versión vectorizada para obtener índices X e Y.
    """
    index = individuo[0]
    x = np.where(index == 1)[0]
    y = np.where(index == 2)[0]
    return x, y


def obtener_matriz_confusion(individuo: np.ndarray, df: pd.DataFrame):
    """
    Calcula la matriz de confusión para un individuo diploide (OPTIMIZADA).
    Usa operaciones vectorizadas de NumPy para mayor velocidad.
    """
    x, y = obtener_x_y(individuo)

    cromosoma = individuo[1]

    df_values = df.values

    if len(x) > 0:
        mask_x = np.all(df_values[:, x] == cromosoma[x], axis=1)
    else:
        mask_x = np.ones(len(df), dtype=bool)
    if len(y) > 0:
        mask_y = np.all(df_values[:, y] == cromosoma[y], axis=1)
    else:
        mask_y = np.ones(len(df), dtype=bool)

    a = np.sum(mask_x & mask_y)
    b = np.sum(~mask_x & mask_y)
    c = np.sum(mask_x & ~mask_y)
    d = np.sum(~mask_x & ~mask_y)

    return [[a, b], [c, d]]

In [ ]:
def support(a: int, N: int) -> float:
    """
    Calcula el soporte de un individuo.
    args:
        a (int): X ∩ Y.
        N (int): Número total de instancias.
    returns:
        float: Soporte del individuo.
    """

    return a / N


def confidence(a: int, c: int) -> float:
    """
    Calcula la confianza de un individuo.
    args:
        a (int): X ∩ Y.
        c (int): X ∩ ~Y.
    returns:
        float: Confianza del individuo.
    """

    if a + c == 0:
        return 0.0
    return a / (a + c)


def lift(a: int, b: int, c: int, N: int) -> float:
    """
    Calcula el lift de un individuo.
    args:
        a (int): X ∩ Y.
        b (int): ~X ∩ Y.
        c (int): X ∩ ~Y.
        N (int): Número total de instancias.
    returns:
        float: Lift del individuo.
    """

    if (a + b) * (a + c) == 0:
        return 0.0
    return (a * N) / ((a + b) * (a + c))


def conviction(a: int, b: int, c: int, d: int, N: int) -> float:
    """
    Calcula la convicción de un individuo.
    args:
        a (int): X ∩ Y.
        b (int): ~X ∩ Y.
        c (int): X ∩ ~Y.
        d (int): ~X ∩ ~Y.
        N (int): Número total de instancias.
    returns:
        float: Convicción del individuo.
    """

    den = 1 - a / (a + c)
    if den == 0:
        return 0.0
    return (1 - (a + b) / N) / den

### Problema de Optimización 1


In [ ]:
def casual_support(a: int, d: int, N: int) -> float:
    """
    Calcula el soporte causal.
    args:
        a (int): X ∩ Y.
        d (int): ~X ∩ ~Y.
        N (int): Número total de instancias.
    returns:
        float: Soporte causal del individuo.
    """

    return (a + d) / N


def casual_confidence(a: int, b: int, c: int, d: int) -> float:
    """
    Calcula la confianza causal.
    args:
        a (int): X ∩ Y.
        b (int): ~X ∩ Y.
        c (int): X ∩ ~Y.
        d (int): ~X ∩ ~Y.
    returns:
        float: Confianza causal del individuo.
    """

    if a + c == 0 or b + d == 0:
        return 0.0
    conf_xy = a / (a + c)

    conf_notx_noty = d / (b + d)

    return 0.5 * (conf_xy + conf_notx_noty)


def max_confidence(a: int, b: int, c: int) -> float:
    """
    Calcula la confianza máxima.
    args:
        a (int): X ∩ Y.
        b (int): ~X ∩ Y.
        c (int): X ∩ ~Y.
        d (int): ~X ∩ ~Y.
    returns:
        float: Confianza máxima del individuo.
    """
    if a + c == 0 or a + b == 0:
        return 0.0
    conf_x_to_y = a / (a + c)

    conf_y_to_x = a / (a + b)

    return max(conf_x_to_y, conf_y_to_x)

### Problema de Optimización 2


In [ ]:
def jaccard(a: int, b: int, c: int, N: int) -> float:
    """
    Calcula el coeficiente de Jaccard de un individuo.
    args:
        a (int): X ∩ Y.
        b (int): ~X ∩ Y.
        c (int): X ∩ ~Y.
        N (int): Número total de instancias.
    returns:
        float: Coeficiente de Jaccard del individuo.
    """

    p_xy = a / N
    p_x = (a + c) / N
    p_y = (a + b) / N

    denominador = p_x + p_y - p_xy

    if denominador == 0:
        return 0.0

    return p_xy / denominador


def cosine(a: int, b: int, c: int, N: int) -> float:
    """
    Calcula la similitud coseno de un individuo.
    args:
        a (int): X ∩ Y.
        b (int): ~X ∩ Y.
        c (int): X ∩ ~Y.
        N (int): Número total de instancias.
    returns:
        float: Similitud coseno del individuo.
    """

    p_x = (a + c) / N
    p_y = (a + b) / N

    if p_x * p_y == 0:
        return 0.0

    p_xy = a / N

    return p_xy / (p_x * p_y) ** 0.5


def phi_coefficient(a: int, b: int, c: int, N: int) -> float:
    """
    Calcula el coeficiente phi de un individuo.
    args:
        a (int): X ∩ Y.
        b (int): ~X ∩ Y.
        c (int): X ∩ ~Y.
        N (int): Número total de instancias.
    returns:
        float: Coeficiente phi del individuo.
    """

    p_xy = a / N

    p_x = (a + c) / N
    p_y = (a + b) / N

    numerador = p_xy - p_x * p_y
    denominador = (p_x * (1 - p_x) * p_y * (1 - p_y)) ** 0.5

    if denominador == 0:
        return 0.0

    return numerador / denominador


def kappa(a: int, b: int, c: int, d: int, N: int) -> float:
    """
    Calcula el coeficiente kappa de un individuo.
    args:
        a (int): X ∩ Y.
        b (int): ~X ∩ Y.
        c (int): X ∩ ~Y.
        d (int): ~X ∩ ~Y.
        N (int): Número total de instancias.
    returns:
        float: Coeficiente kappa del individuo.
    """

    p_xy = a / N

    p_notx_noty = d / N

    p_x = (a + c) / N
    p_y = (a + b) / N
    p_notx = (b + d) / N
    p_noty = (c + d) / N

    numerador = p_xy + p_notx_noty - p_x * p_y - p_notx * p_noty
    denominador = 1 - p_x * p_y - p_notx * p_noty

    if denominador == 0:
        return 0.0

    return numerador / denominador

### Función general


In [ ]:
def obtener_objetivos(
    individuo: np.ndarray, df: pd.DataFrame, problema: int = 0
) -> np.ndarray:
    """ """
    matriz_confusion = obtener_matriz_confusion(individuo, df)
    a = matriz_confusion[0][0]
    b = matriz_confusion[0][1]
    c = matriz_confusion[1][0]
    d = matriz_confusion[1][1]
    N = df.shape[0]
    if problema == 0:
        return np.array(
            [
                -support(a, N),
                -confidence(a, c),
                -lift(a, b, c, N),
                -conviction(a, b, c, d, N),
            ]
        )
    elif problema == 1:
        return np.array(
            [
                -casual_support(a, d, N),
                -casual_confidence(a, b, c, d),
                -max_confidence(a, b, c),
            ]
        )
    elif problema == 2:
        return np.array(
            [
                -jaccard(a, b, c, N),
                -cosine(a, b, c, N),
                -phi_coefficient(a, b, c, N),
                -kappa(a, b, c, d, N),
            ]
        )

In [ ]:
def obtener_objetivos_poblacion(
    poblacion: np.ndarray, df: pd.DataFrame, problema: int = 0
) -> np.ndarray:
    return np.array(
        [obtener_objetivos(individuo, df, problema) for individuo in poblacion]
    )

### Estadísticas


In [ ]:
def obtener_frente_pareto(objetivos_matriz: np.ndarray) -> np.ndarray:
    """
    Extrae el frente de Pareto (índices no dominados) de una matriz de objetivos.

    args:
        objetivos_matriz: Matriz (n_individuos, m_objetivos) con valores ya calculados
        es_maximizacion: Lista indicando si cada objetivo se maximiza

    returns:
        indices_pareto: Índices booleanos de individuos en el frente de Pareto
        objetivos_pareto: Valores de objetivos del frente de Pareto
    """
    n, m = objetivos_matriz.shape
    objetivos_matriz = objetivos_matriz * -1
    es_pareto = np.ones(n, dtype=bool)

    for i in range(n):
        if not es_pareto[i]:
            continue

        for j in range(n):
            if i == j or not es_pareto[j]:
                continue

            # Verificar si j domina a i
            domina = True
            al_menos_uno_mejor = False

            for k in range(m):
                if objetivos_matriz[j, k] < objetivos_matriz[i, k]:
                    domina = False
                    break
                if objetivos_matriz[j, k] > objetivos_matriz[i, k]:
                    al_menos_uno_mejor = True

            if domina and al_menos_uno_mejor:
                es_pareto[i] = False
                break

    objetivos_pareto = objetivos_matriz[es_pareto]

    return es_pareto, objetivos_pareto


def _hipervolumen_wfg(objetivos: np.ndarray, ref: np.ndarray) -> float:
    """Algoritmo WFG para n dimensiones."""
    if len(objetivos) == 0:
        return 0.0

    m = objetivos.shape[1]

    indices = np.argsort(objetivos[:, -1])
    objetivos_sorted = objetivos[indices]

    hv = 0.0
    ref_actual = ref.copy()

    for i in range(len(objetivos_sorted)):
        punto = objetivos_sorted[i]
        altura = ref_actual[-1] - punto[-1]

        if altura <= 0:
            continue

        objetivos_rest = objetivos_sorted[i:, :-1]
        ref_rest = ref_actual[:-1].copy()

        for j in range(len(ref_rest)):
            ref_rest[j] = min(ref_rest[j], punto[j])

        hv_rest = _hipervolumen_wfg(objetivos_rest, ref_rest)
        hv += altura * hv_rest

        ref_actual[-1] = punto[-1]

    return hv

In [ ]:
def df_resumen_gen(objetivos: np.ndarray, df: pd.DataFrame, gen: int) -> pd.DataFrame:
    valores = []
    for i in range(len(df.columns) - 1):
        min_val = -np.max(objetivos[:, i])
        max_val = -np.min(objetivos[:, i])
        promedio = -np.mean(objetivos[:, i])
        std = np.std(objetivos[:, i])
        valores.append([min_val, max_val, std, promedio])
    objetivos_pareto = obtener_frente_pareto(objetivos)
    valores.append(
        _hipervolumen_wfg(objetivos_pareto[1], np.zeros(len(objetivos_pareto[0])))
    )
    df.loc[len(df)] = valores + [gen]
    return df

## Funciones Clásicas de AG


### Selección de Padres


In [ ]:
def seleccionar_padres(poblacion: np.ndarray, q_padres: int) -> np.ndarray:
    """
    Selecciona padres de individuos de la población.
    args:
        poblacion (np.ndarray): Población de shape (n_individuos, 2, n_genes)
        q_padres (int): Cantidad de padres a seleccionar.
    Returns:
        np.ndarray: Array de shape (q_padres, 2, n_genes)
    """
    n_parejas_disponibles = poblacion.shape[0]

    indices = np.random.choice(n_parejas_disponibles, size=q_padres, replace=False)

    return poblacion[indices]

### Cruza n-puntos


In [ ]:
def cruza_n_puntos(padre1: np.ndarray, padre2: np.ndarray, n: int = 3):
    """
    Realiza una cruza de n puntos entre dos padres.
    args:
        padre1 (np.ndarray): Padre 1 de shape (2, n_genes)
        padre2 (np.ndarray): Padre 2 de shape (2, n_genes)
        n (int): Cantidad de puntos de cruza. Default es 3.
    returns:
        (np.ndarray, np.ndarray): 2 Hijos de shape (2, n_genes)
    """
    interseccion = int(padre1.shape[1] / n)
    hijo1 = padre1.copy()
    hijo2 = padre2.copy()

    for i in range(n):
        if i % 2 == 0:
            continue
        elif i == n - 1:
            hijo1[:, interseccion * i :] = padre2[:, interseccion * i :]
            hijo2[:, interseccion * i :] = padre1[:, interseccion * i :]
        else:
            hijo1[:, interseccion * i : interseccion * (i + 1)] = padre2[
                :, interseccion * i : interseccion * (i + 1)
            ]
            hijo2[:, interseccion * i : interseccion * (i + 1)] = padre1[
                :, interseccion * i : interseccion * (i + 1)
            ]

    return hijo1, hijo2


def reproduccion(padres: np.ndarray):
    """ """
    hijos = []
    for i in range(0, len(padres), 2):
        hijo1, hijo2 = cruza_n_puntos(padres[i], padres[i + 1])
        hijos.append(hijo1)
        hijos.append(hijo2)
    return np.array(hijos)

### Mutación


In [ ]:
def mutacion_extension(
    individuo: np.ndarray,
    df: pd.DataFrame = df_dummy,
):
    indice = np.random.randint(0, individuo.shape[1])
    individuo[0][indice] = np.random.choice([0, 1])
    individuo[1][indice] = np.random.choice(df[df_dummy.columns[indice]])
    return individuo

In [ ]:
def mutacion_contraccion(individuo: np.ndarray):
    indice = np.random.randint(0, individuo.shape[1])
    individuo[0][indice] = 0
    individuo[1][indice] = 0
    return individuo

In [ ]:
def mutacion_reemplazo(
    individuo: np.ndarray,
    df: pd.DataFrame = df_dummy,
):
    indice = np.random.randint(0, individuo.shape[1])
    individuo[1][indice] = np.random.choice(df[df_dummy.columns[indice]])
    return individuo

In [ ]:
def mutacion(
    individuo: np.ndarray, porcentaje: float = 0.1, df: pd.DataFrame = df_dummy
):
    """
    Función que muta la población.
    """
    if np.random.random() < porcentaje:
        individuo = mutacion_extension(individuo, df)
    elif np.random.random() < porcentaje:
        individuo = mutacion_contraccion(individuo)
    elif np.random.random() < porcentaje:
        individuo = mutacion_reemplazo(individuo, df)
    return individuo

## Main


In [ ]:
def generacion_AG(
    poblacion: np.ndarray,
    vecindarios: np.ndarray,
    objetivos: np.ndarray,
    evaluar_con_agregacion: Callable,
    z: np.ndarray,
    pesos: np.ndarray,
    df: pd.DataFrame = df_dummy,
    problema: int = 1,
    porcentaje_mutacion: float = 0.1,
    porcentaje_cruza: float = 0.1,
):
    for i in range(len(poblacion)):
        vecindario = vecindarios[i]
        padres = seleccionar_padres(poblacion[vecindario], 2)
        if np.random.random() < porcentaje_cruza:
            hijo, _ = cruza_n_puntos(padres[0], padres[1])
        else:
            hijo = padres[0]
        hijo_mutado = mutacion(hijo, porcentaje_mutacion, df)
        objetivos_hijo_mutado = obtener_objetivos(hijo_mutado, df, problema)
        z = actualizar_punto_ideal(z, hijo_mutado, objetivos_hijo_mutado)
        for j in vecindario:
            if comparar_individuos(
                objetivos_hijo_mutado, objetivos[j], pesos[j], z, evaluar_con_agregacion
            ):
                poblacion[j] = hijo_mutado.copy()
                objetivos[j] = objetivos_hijo_mutado
    return poblacion, objetivos, z, pesos

In [ ]:
def moead(
    df: pd.DataFrame,
    evaluar_con_agregacion: Callable = tchebycheff,
    problema: int = 1,
    N: int = 120,
    n_generaciones: int = 100,
    porcMuta: float = 0.5,
    porcCruza: float = 0.5,
):
    poblacion = generar_poblacion(df, N)
    if problema == 1:
        M = 3
    else:
        M = 4
    cols = ["obj_{i}".format(i=i) for i in range(M)] + ["hipervolumen", "generacion"]
    df_resultados = pd.DataFrame(columns=cols)
    pesos = Dan_Dennis(poblacion, M)
    vecindarios = encontrar_vecindarios(pesos, 3)
    objetivos = obtener_objetivos_poblacion(poblacion, df, problema)
    z = inicializar_punto_ideal(poblacion, objetivos, problema)
    for gen in tqdm(range(n_generaciones)):
        poblacion, objetivos, z, pesos = generacion_AG(
            poblacion,
            vecindarios,
            objetivos,
            evaluar_con_agregacion,
            z,
            pesos,
            df,
            problema,
            porcMuta,
            porcCruza,
        )
        if (gen + 1) % 50 == 0:
            df_resultados = df_resumen_gen(objetivos, df_resultados, gen)
            df_resultados.to_csv("moead.csv")
    return poblacion, objetivos, df_resultados

In [ ]:
poblacion, objetivos, df_resultados = moead(
    df_dummy, tchebycheff, problema=1, n_generaciones=300
)

  1%|▏         | 4/300 [16:05<19:51:19, 241.48s/it]


KeyboardInterrupt: 

In [ ]:
df_resultados.to_csv("moead.csv")